<a href="https://colab.research.google.com/github/skyler14/ICIAR-histology/blob/master/APM_Base_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Version 1 - Base Code to Setup Environment Space on Collab

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# installing dependencies
import os
import pandas as pd
import numpy as np
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf

In [0]:
# set our working path and dataset dir

from pathlib import Path
ROOT_DIR = Path(os.getcwd())
TRAIN_IMAGE_DIR = ROOT_DIR / "img"


In [15]:
# first attempt to download all images

# 1. choose a local (colab) directory to store the data.
local_download_path = TRAIN_IMAGE_DIR
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1q_DHHFO4JJ9M6mCndclnaer5XHcFNdOd' in parents"}).GetList()


# when we start developing the ability to read the microscope files we'll swap 
# it out with this and nest our loop, example with test dataset below
# folder list order : Photos/, WSI/, WSI/thumbnails/
# file_list = [drive.ListFile(
#     {'q': "'1zYUjNV4EPxQmcU32i-xIWZbBBmuwpu_i' in parents"}).GetList(),drive.ListFile(
#     {'q': "'1mCwOqg3qE6kXwmuM1RGB2ngIsZ10UBXD' in parents"}).GetList(),drive.ListFile(
#     {'q': "'1c1xB68siz2PtXwMJ80OAKFGIxPGvc9DS' in parents"}).GetList()]


# WARNING: make sure to match these two lists in any updates
folder_list = [drive.ListFile(
    {'q': "'1q_DHHFO4JJ9M6mCndclnaer5XHcFNdOd' in parents"}).GetList(),drive.ListFile(
    {'q': "'1nMWsOO7nf-rzknjnm9kp5qAd6NvJ02r9' in parents"}).GetList(),drive.ListFile(
    {'q': "'1JUBneAIR0nXtNBqHS6WaDtqXGmxnchLn' in parents"}).GetList(),drive.ListFile(
    {'q': "'1UJT-K0IpCW7rUUq4BuYzHgqyds_xs073' in parents"}).GetList()]

folder_names = ['Benign','InSitu',"Invasive","Normal"]

print(file_list)
for folder,dirname in zip(folder_list,folder_names):
  n=0
  os.mkdir(local_download_path / dirname)
  for f in folder:
    # n-capped for the time being so you dont sit waiting for downloads, 
    # set n to 1000 or whatever to dl everything 
    n=n+1
    if n<3:
      try:
        # 3. Create & download by id.
        print('title: %s, id: %s' % (f['title'], f['id']))
        fname = (local_download_path / dirname / f['title'])
        print('downloading to {}'.format(fname))
        f_ = drive.CreateFile({'id': f['id']})
        f_.GetContentFile(fname)
      except:
        print("item ",n,"failed")

[GoogleDriveFile({'kind': 'drive#file', 'id': '13-pxwAEUTS_S8N44UO5154v38GW7mnC0', 'etag': '"ARKMDIyi8Qt0WnRZL7i9hjeAaoo/MTU3MjgwMjYzODQyNw"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/13-pxwAEUTS_S8N44UO5154v38GW7mnC0', 'webContentLink': 'https://drive.google.com/a/utexas.edu/uc?id=13-pxwAEUTS_S8N44UO5154v38GW7mnC0&export=download', 'alternateLink': 'https://drive.google.com/a/utexas.edu/file/d/13-pxwAEUTS_S8N44UO5154v38GW7mnC0/view?usp=drivesdk', 'embedLink': 'https://drive.google.com/a/utexas.edu/file/d/13-pxwAEUTS_S8N44UO5154v38GW7mnC0/preview?usp=drivesdk', 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/image/tiff', 'thumbnailLink': 'https://lh3.googleusercontent.com/DJV7XpKUUWyHZyVfU2m0PEazOz-aHKtbztNnjOEjT91K0vkrUM82pwy_BDe4afCB8Llu19EG714=s220', 'title': 'b100.tif', 'mimeType': 'image/tiff', 'labels': {'starred': False, 'hidden': False, 'trashed': False, 'restricted': False, 'viewed': False}, 'copyRequiresWriterPermission': False, 'createdDate'

SyntaxError: ignored

In [40]:
# for reference with regards to directory walking

cell_df = pd.DataFrame(columns=['class', 'image', 'train/test?'])
# print(cell_df)


imglist=[]
for paths, dirs, files in os.walk(TRAIN_IMAGE_DIR):
    # print(dirs)
    for f in files:
        filename = os.path.join(paths,f)
# #     read here, some useful commands to get an idea of what os.walk outputs
        imglist.append(filename)
        # print(filename, dirs)
        # print(os.path.dirname(filename))
        # path = os.path.dirname(filename)
        # os.chdir(path)


# tada, here's a list of all files 
print(imglist)


['/content/img/b068.tif', '/content/img/b076.tif', '/content/img/b040.tif', '/content/img/b072.tif', '/content/img/b018.tif', '/content/img/b014.tif', '/content/img/b091.tif', '/content/img/b064.tif', '/content/img/b092.tif', '/content/img/b045.tif', '/content/img/b087.tif', '/content/img/b060.tif', '/content/img/b093.tif', '/content/img/b041.tif', '/content/img/b001.tif', '/content/img/iv099.tif', '/content/img/b004.tif', '/content/img/b007.tif', '/content/img/b036.tif', '/content/img/b013.tif', '/content/img/b081.tif', '/content/img/b079.tif', '/content/img/b063.tif', '/content/img/b028.tif', '/content/img/b021.tif', '/content/img/b044.tif', '/content/img/b100.tif', '/content/img/b009.tif', '/content/img/b097.tif', '/content/img/b082.tif', '/content/img/b008.tif', '/content/img/b006.tif', '/content/img/b055.tif', '/content/img/b050.tif', '/content/img/b020.tif', '/content/img/b049.tif', '/content/img/b066.tif', '/content/img/b005.tif', '/content/img/b052.tif', '/content/img/b058.tif'

In [0]:
from google.colab import drive
drive.mount('/content/drive')